In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import json
import ast

In [ ]:
# read files
#person_id,id,name,character,role
df_credits = pd.read_csv('data/credits.csv')
#id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
df_netflix = pd.read_csv('data/titles.csv')
df_netflix = df_netflix[(df_netflix['imdb_score'].notna() | df_netflix['tmdb_score'].notna())]

In [ ]:
# basic statistics (for milestone 1)
'''
TODO:
- Milestone 1:
    - number of movies: year, budget, language
    - number of actors: total appearances in movies 
    - number of directors
- Milestone 2 & 3:
'''

def sort_json(data):
    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True)}

def plot_quantities(data, color, xlabel, ylabel, title):
    x = list(data.keys())
    y = list(data.values())

    plt.figure(figsize=(15, 10))
    plt.bar(x, y, color=color, width=0.8)
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.xticks(rotation=60)
    plt.title(title)
    
    plt.show()

# number of movies/series
list_movie_ids = df_netflix['id'].unique().tolist()
print('Number of netflix movies/series:', len(list_movie_ids))
print('Number of shows:', len(df_netflix[(df_netflix['type'] == 'SHOW')]))
print('Number of movies:', len(df_netflix[(df_netflix['type'] == 'MOVIE')]))

# years
df_years = df_netflix['release_year']
years = {}
for year in df_years:
    if year in years:
        years[year] += 1
    else:
        years[year] = 1
years = sort_json(years)
print('Number of years:', len(years.keys()))

# genre
df_genres = df_netflix['genres']
genres = {}
for row in df_genres:
    genres_movie = ast.literal_eval(row) 
    for genre in genres_movie:
        if genre in genres:
            genres[genre] += 1
        else:
            genres[genre] = 1
genres = sort_json(genres)
print('Number of genres:', len(genres.keys()))

# age certification
df_age_certification = df_netflix['age_certification']
age_certifications = {}
for cert in df_age_certification:
    cert = str(cert) 
    cert = cert if cert != 'nan' else 'No Age Certification'
    
    if cert in age_certifications:
        age_certifications[cert] += 1
    else:
        age_certifications[cert] = 1
age_certifications = sort_json(age_certifications)
print('Number of age certifications:', len(age_certifications.keys()))

# production countries
df_production_countries = df_netflix['production_countries']
production_countries = {}
for row in df_production_countries:
    countries = ast.literal_eval(row) 
    #json.loads(row)
    for country in countries:
        if country in production_countries:
            production_countries[country] += 1
        else:
            production_countries[country] = 1
production_countries = sort_json(production_countries)
print('Number of production countries:', len(production_countries.keys()))
n_min = 30
production_countries = {k: v for k, v in production_countries.items() if v >= n_min}

# seasons for series
filter_series = df_netflix['type'] == 'SHOW'
df_seasons = df_netflix[filter_series]['seasons']
seasons = {}
for row in df_seasons:
    season = str(int(row))
    if season in seasons:
        seasons[season] += 1
    else:
        seasons[season] = 1
seasons = sort_json(seasons)


# imdb score
df_imdb_scores = df_netflix['imdb_score']
imdb_scores = []
for row in df_imdb_scores:
    imdb_scores.append(row)
df_tmdb_scores = df_netflix['tmdb_score']
tmdb_scores = []
for row in df_tmdb_scores:
    tmdb_scores.append(row)

plt.figure(figsize=(15, 10))
plt.scatter(imdb_scores, tmdb_scores, color='cyan')

plt.xlabel('IMDB scores')
plt.ylabel('TMDB scores')

plt.title('IMDB/TMDB scores')

plt.show()

plot_quantities(years, 'blue', 'Year', 'Number of movies/series', 'Number of Netflix movies/series per year')
plot_quantities(genres, 'maroon', 'Genre', 'Number of movies/series', 'Number of Netflix movies/series per genre')
plot_quantities(age_certifications, 'green', 'Age Certification', 'Number of movies/series', 'Number of Netflix movies/series per age certification')
plot_quantities(production_countries, 'orange', 'Production Countries', 'Number of movies/series', f'Number of Netflix movies/series per production country (at least {n_min} movies)')
plot_quantities(seasons, 'red', 'Seasons', 'Number of series', 'Number of Netflix series per season')

In [ ]:
# number of actors
print('Number of actors:', len(df_credits[df_credits['role'] == 'ACTOR']['person_id'].unique()))
# number of directors
print('Number of directors:', len(df_credits[df_credits['role'] == 'DIRECTOR']['person_id'].unique()))